In [21]:
import pandas as pd
import os
from google.colab import files

In [22]:
# Conectando e baixando arquivos do GitHub

arquivos = ["DadosBase2023.csv", "DadosBase2024.csv", "SH.csv", "NomesMunicipios.csv", "Paises.csv",]
url_base = "https://raw.githubusercontent.com/Lucasdz-7/Projeto-Integrador-1/main/Projeto/"


In [23]:
# Criando Dataframes

Exportacao23 = pd.read_csv(url_base + 'DadosBase2023.csv', low_memory=False, sep=';', encoding='UTF-8')
Exportacao24 = pd.read_csv(url_base + 'DadosBase2024.csv', low_memory=False, sep=';', encoding='UTF-8')
SH = pd.read_csv(url_base + 'SH.csv', low_memory=False, sep=';', encoding='ISO-8859-1')
Municipios = pd.read_csv(url_base + 'NomesMunicipios.csv', low_memory=False, sep=';', encoding='ISO-8859-1')
Paises = pd.read_csv(url_base + 'Paises.csv', low_memory=False, sep=';', encoding='ISO-8859-1')

In [24]:
# Juntando dados de exportação

ExportacaoTotal = pd.concat([Exportacao23, Exportacao24],ignore_index=False)
ExportacaoTotal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2266180 entries, 0 to 1144827
Data columns (total 8 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_ANO      int64 
 1   CO_MES      int64 
 2   SH4         int64 
 3   CO_PAIS     int64 
 4   SG_UF_MUN   object
 5   CO_MUN      int64 
 6   KG_LIQUIDO  int64 
 7   VL_FOB      int64 
dtypes: int64(7), object(1)
memory usage: 155.6+ MB


In [ ]:
ExportacaoTotal.head(5)

In [ ]:
Municipios.head(10)

In [25]:
# Tratando os dados do arquivo de municípios
MunicipiosSp = Municipios.query("SG_UF == 'SP'")
MunicipiosSp.rename(columns={"CO_MUN_GEO": "CO_MUN"}, inplace=True)
MunicipiosSp.head(5)

<ipython-input-25-58dda61300e0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MunicipiosSp.rename(columns={"CO_MUN_GEO": "CO_MUN"}, inplace=True)


,CO_MUN,NO_MUN,NO_MUN_MIN,SG_UF
1130,3446900,SANTA LUCIA,Santa Lúcia,SP
1131,3447007,SANTA MARIA DA SERRA,Santa Maria da Serra,SP
1132,3447106,SANTA MERCEDES,Santa Mercedes,SP
1133,3447205,SANTANA DA PONTE PENSA,Santana da Ponte Pensa,SP
1134,3447304,SANTANA DE PARNAIBA,Santana de Parnaíba,SP


In [26]:
# Tratando dados do arquivo de Exportação para visualização apenas do estado de São Paulo

ExportacaoSp = ExportacaoTotal.query("SG_UF_MUN == 'SP'")
ExportacaoSp.head(5)

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
1,2023,11,4009,493,SP,3456701,473,14008
2,2023,11,8536,365,SP,3438709,2,181
5,2023,6,8421,63,SP,3450308,6002,131714
6,2023,4,9403,586,SP,3447304,70,264
9,2023,6,8419,158,SP,3409502,44,4456


In [27]:
# Juntando arquivos de municípios e de Exportação

ExportacaoSp = ExportacaoSp.merge(MunicipiosSp[['CO_MUN','NO_MUN_MIN']], on = 'CO_MUN', how = 'left')
ExportacaoSp.head(5)

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,NO_MUN_MIN
0,2023,11,4009,493,SP,3456701,473,14008,Vinhedo
1,2023,11,8536,365,SP,3438709,2,181,Piracicaba
2,2023,6,8421,63,SP,3450308,6002,131714,São Paulo
3,2023,4,9403,586,SP,3447304,70,264,Santana de Parnaíba
4,2023,6,8419,158,SP,3409502,44,4456,Campinas


In [ ]:
# Renomeando colunas

ExportacaoSp.drop(columns=['CO_MUN'], inplace=True)
ExportacaoSp.rename(columns={'NO_MUN_MIN': 'Nome_Municipio'}, inplace=True)
ExportacaoSp.head(10)

In [ ]:
SH.head(5)

In [29]:
# Limpando os dados do arquivo SH que são desnecessários

SH.drop(columns=['CO_SH6', 'NO_SH6_POR',	'NO_SH6_ESP',	'NO_SH6_ING', 'NO_SH4_ESP', 'NO_SH4_ING',
'CO_SH2',	'NO_SH2_POR',	'NO_SH2_ESP',	'NO_SH2_ING',	'CO_NCM_SECROM',	'NO_SEC_POR',
'NO_SEC_ESP',	'NO_SEC_ING'], inplace=True)
SH.rename(columns={'CO_SH4': 'SH4'}, inplace=True)
SH = SH.drop_duplicates(subset='SH4')


SH.head(5)

,SH4,NO_SH4_POR
0,101,"Cavalos, asininos e muares, vivos"
8,102,Animais vivos da espécie bovina
14,103,Animais vivos da espécie suína
17,104,Animais vivos das espécies ovina e caprina
19,105,"Galos, galinhas, patos, gansos, perus, peruas ..."


In [30]:
# Juntando os dados do arquivo de SH com o arquivo de exportações em São Paulo.

ExportacaoSp = ExportacaoSp.merge(SH[['SH4','NO_SH4_POR']], on = 'SH4', how = 'left')
ExportacaoSp.head(5)

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,KG_LIQUIDO,VL_FOB,Nome_Municipio,NO_SH4_POR
0,2023,11,4009,493,SP,473,14008,Vinhedo,"Tubos de borracha vulcanizada não endurecida, ..."
1,2023,11,8536,365,SP,2,181,Piracicaba,"Aparelhos para interrupção, seccionamento, pro..."
2,2023,6,8421,63,SP,6002,131714,São Paulo,"Centrifugadores, incluídos os secadores centrí..."
3,2023,4,9403,586,SP,70,264,Santana de Parnaíba,Outros móveis e suas partes
4,2023,6,8419,158,SP,44,4456,Campinas,"Aparelhos e dispositivos, mesmo aquecidos elec..."


In [ ]:
# Renomeando colunas

ExportacaoSp.drop(columns=['SH4'], inplace=True)
ExportacaoSp.rename(columns={'NO_SH4_POR': 'Produto_SH4'}, inplace=True)
ExportacaoSp.head(10)

In [ ]:
Paises.head(5)

In [32]:
# Limpando o arquivo que contém a informação do nome dos países de acordo com seu código

Paises.drop(columns=['CO_PAIS_ISON3', 'CO_PAIS_ISOA3', 'NO_PAIS_ING', 'NO_PAIS_ESP'], inplace=True)
Paises.head(5)

,CO_PAIS,NO_PAIS
0,0,Não Definido
1,13,Afeganistão
2,15,"Aland, Ilhas"
3,17,Albânia
4,20,"Alboran-Perejil, Ilhas"


In [33]:
# Juntando os dados do arquivo de países com o de exportação

ExportacaoSp = ExportacaoSp.merge(Paises[['CO_PAIS','NO_PAIS']], on = 'CO_PAIS', how = 'left')

In [ ]:
# Renomeando colunas

ExportacaoSp.drop(columns=['CO_PAIS'], inplace=True)
ExportacaoSp.rename(columns={'NO_PAIS': 'PAIS_DESTINO'}, inplace=True)
ExportacaoSp.head(10)

In [35]:
# Criando a coluna de valor agregado, usando os valores de FOB e KG, delimitando até 2 casas depois da vírgula, o if garante que mesmo que exista um valor 0 no KG a resposta não seja um erro

ExportacaoSp["Valor_Agregado"] = ExportacaoSp.apply(
    lambda row: round(row["VL_FOB"] / row["KG_LIQUIDO"], 2) if row["KG_LIQUIDO"] > 0 else 0, axis=1
)

# Sobrescrevendo em cima do arquivo base
ExportacaoSp.to_csv("ExportacaoSP.csv", sep=";", index=False, encoding="utf-8")

ExportacaoSp.head(10)

,CO_ANO,CO_MES,SG_UF_MUN,KG_LIQUIDO,VL_FOB,Nome_Municipio,Produto_SH4,PAIS_DESTINO,Valor_Agregado
0,2023,11,SP,473,14008,Vinhedo,"Tubos de borracha vulcanizada não endurecida, ...",México,29.62
1,2023,11,SP,2,181,Piracicaba,"Aparelhos para interrupção, seccionamento, pro...",Indonésia,90.50
2,2023,6,SP,6002,131714,São Paulo,"Centrifugadores, incluídos os secadores centrí...",Argentina,21.95
3,2023,4,SP,70,264,Santana de Parnaíba,Outros móveis e suas partes,Paraguai,3.77
4,2023,6,SP,44,4456,Campinas,"Aparelhos e dispositivos, mesmo aquecidos elec...",Chile,101.27
5,2023,9,SP,372,8184,Vinhedo,Preparações capilares,Tunísia,22.00
6,2023,6,SP,0,6,Porto Feliz,Outras obras de ferro ou aço,Equador,0.00
7,2023,12,SP,1473,4741262,São Paulo,Máquinas automáticas para processamento de dad...,Estados Unidos,3218.78
8,2023,9,SP,60,604,São Paulo,Preparações capilares,Argélia,10.07
9,2023,1,SP,116,6344,Sumaré,"Livros de registo e de contabilidade, blocos d...",Chile,54.69


In [36]:
# Contar valores zerados no ExportacaoSp
contagem_zeros_sp = (ExportacaoSp == 0).sum()

# Filtrando dados
colunas_relevantes = ['CO_ANO', 'CO_MES', 'SG_UF_MUN', 'KG_LIQUIDO', 'VL_FOB', 'Nome_Municipio', 'Produto_SH4', 'PAIS_DESTINO',]
contagem_zeros_sp = (ExportacaoSp[colunas_relevantes] == 0).sum()

# Criar uma tabela com os resultados
tabela_zeros_sp = pd.DataFrame(contagem_zeros_sp).reset_index()
tabela_zeros_sp.columns = ['Coluna', 'Qtd_Valores_Zerados']

print(tabela_zeros_sp)

           Coluna  Qtd_Valores_Zerados
0          CO_ANO                    0
1          CO_MES                    0
2       SG_UF_MUN                    0
3      KG_LIQUIDO                88158
4          VL_FOB                 2453
5  Nome_Municipio                    0
6     Produto_SH4                    0
7    PAIS_DESTINO                    0


In [38]:
# Deletando linhas com qualquer valor zerado em alguma coluna

ExportacaoSp = ExportacaoSp[(ExportacaoSp != 0).all(axis=1)]
(ExportacaoSp == 0).sum()

In [42]:
# Salvando o arquivo principal
ExportacaoSp.to_csv("ExportacaoSP.csv", sep=";", index=False, encoding="utf-8")

print(os.listdir())
files.download("ExportacaoSP.csv")

['.config', 'ExportacaoSP.csv', 'sample_data']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Salvando o arquivo de zeros
tabela_zeros_sp.to_csv("tabelas_zeros_sp.csv", sep=";", index=False, encoding="utf-8")

print(os.listdir())
files.download("tabelas_zeros_sp.csv")